### Load packages
First we need to load the python packages that are required to perform the analysis

In [1]:
%matplotlib inline
import os
import rasterio
import numpy 
import pandas
import scipy.io # to load the matlab files
from tqdm import tqdm
from DamageScanner import DamageScanner # the tool to estimate the losses

data_path = os.path.join('..','Data')

In [2]:
# function to load DDM data
def read_DDM_input(DDM_data_path):
    DDM_data = {}
    DDM_data['Curves_structure'] = pandas.read_excel(DDM_data_path,sheet_name='curves_structure').values
    DDM_data['Curves_content'] = pandas.read_excel(DDM_data_path,sheet_name='curves_content').values
    DDM_data['MaxDam_structure'] = pandas.read_excel(DDM_data_path,sheet_name='maxdam_structure').values
    DDM_data['MaxDam_content'] = pandas.read_excel(DDM_data_path,sheet_name='maxdam_content').values  
    
    return DDM_data

### Create a function to calculate the risk

In [3]:
def risk(DDM_data_path,timeframe='current'):

    # Set paths to data and load initial data
    DDM_data = read_DDM_input(DDM_data_path) # the depth-damage curves and maximum damages
    landuse_map = os.path.join(data_path,'landuse.asc') # the land-use map
    inundation_maps = scipy.io.loadmat(os.path.join(data_path,'Waterdepth_{}.mat'.format(timeframe))) # the inundation files for the current situation
 
    return_periods = [10,100,1000,2000,4000,10000]
    
    damages = []
    for return_period in tqdm(return_periods):
        if timeframe == 'current':
            inun_map = 'a0_{}'.format(return_period)
        elif timeframe == '2100':
            inun_map = 'a2_{}'.format(return_period)
            
        damage_structure,damagebin_structure = DamageScanner(landuse_map,inundation_maps[inun_map],DDM_data['Curves_structure'],
                                                         DDM_data['MaxDam_structure'])

        # run the DamageScanner for the content
        damage_content,damagebin_content = DamageScanner(landuse_map,inundation_maps[inun_map],DDM_data['Curves_content'],
                                                             DDM_data['MaxDam_content'])
        
        damages.append(damage_content+damage_structure)
        
    return numpy.trapz(damages[::-1],[1/x for x in return_periods][::-1])

### Estimate avoided losses

Calculate current and future risk using 'normal curves'

In [4]:
DDM_data_path = os.path.join(data_path,'DDM_data.xlsx')
current_risk_no_adapt = risk(DDM_data_path,timeframe='current')
future_risk_no_adapt = risk(DDM_data_path,timeframe='2100')

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:02<00:00,  2.43it/s]


Calculate current and future risk using 'changed curves'

In [5]:
DDM_data_path = os.path.join(data_path,'DDM_data.xlsx')
current_risk_adapt = risk(DDM_data_path,timeframe='current')
future_risk_adapt = risk(DDM_data_path,timeframe='2100')

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:02<00:00,  2.62it/s]


Calculate avoided losses for current and future risk

In [52]:
current_avoided_losses = current_risk_no_adapt-current_risk_adapt
future_avoided_losses = future_risk_no_adapt-future_risk_adapt

### Estimate total avoided losses over the next 50 years for the current and future situation

In [38]:
time_horizon = XXX # fill in time horizon
discountRate = XXX # fill in discountrate

# and we use the numpy net present value function:
total_avoided_losses_current = numpy.npv(discountRate, [current_avoided_losses]*time_horizon)
total_avoided_losses_future = numpy.npv(discountRate, [future_avoided_losses]*time_horizon)

### Estimate total houses that need to be adapted

Let's load the landuse map first

In [27]:
with rasterio.open(os.path.join(data_path,'landuse.asc')) as src:
    landuse = src.read()[0,:,:]
    cellsize = src.res[0]*src.res[1]

And get the total landuse for each landuse class

In [29]:
uniques,counts = numpy.unique(landuse,return_counts=True) # first count the cells 

df_landuse = pandas.DataFrame.from_dict(dict(zip(uniques, counts)),
                                        orient='index',columns=['cell_count']) # turn into a dataframe

df_landuse['area'] = df_landuse.cell_count*cellsize # and multiply with the cellsize to get the total area

Now we know the total area of houses (land-use class 111). Estimate a sensible average area for each house, to estimate the total amount of residential buildings within the Heijplaat 

In [42]:
total_houses = df_landuse.loc[111,'area']/XXX # fill in average area of each house

### Estimate benefit-cost ratio for current and future situation

In [48]:
bca_current = total_avoided_losses_current/(total_houses*XXX) # fill in investment value

In [49]:
bca_future = total_avoided_losses_future/(total_houses*XXX) # fill in investment value

### Estimate optimal investment value for current situation